In [1]:
!apt-get install openjdk-11-jdk -y
!pip install pyspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxt-dev libxtst6 libxxf86dga1 openjdk-11-jre
  x11-utils
Suggested packages:
  libxt-doc openjdk-11-demo openjdk-11-source visualvm mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxt-dev libxtst6 libxxf86dga1 openjdk-11-jdk
  openjdk-11-jre x11-utils
0 upgraded, 10 newly installed, 0 to remove and 35 not upgraded.
Need to get 6,920 kB of archives.
After this operation, 16.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-core all 2.37-2build1 [1,041 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-extra all 2.37-2build1 [2,041 kB]
Get:3 http://archive.ubuntu.com/ubuntu jam

In [6]:
#SparkSession
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("iFood_ABTest_Case") \
    .getOrCreate()

print("✅ PySpark configurado e SparkSession criada!")

✅ PySpark configurado e SparkSession criada!


In [7]:
#consumidores
!wget https://data-architect-test-source.s3-sa-east-1.amazonaws.com/consumer.csv.gz -O consumer.csv.gz


--2025-06-17 20:46:34--  https://data-architect-test-source.s3-sa-east-1.amazonaws.com/consumer.csv.gz
Resolving data-architect-test-source.s3-sa-east-1.amazonaws.com (data-architect-test-source.s3-sa-east-1.amazonaws.com)... 52.95.165.110, 3.5.232.123, 3.5.232.21, ...
Connecting to data-architect-test-source.s3-sa-east-1.amazonaws.com (data-architect-test-source.s3-sa-east-1.amazonaws.com)|52.95.165.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48501207 (46M) [text/csv]
Saving to: ‘consumer.csv.gz’

consumer.csv.gz     100%[===================>]  46.25M  13.4MB/s    in 4.0s    

2025-06-17 20:46:38 (11.7 MB/s) - ‘consumer.csv.gz’ saved [48501207/48501207]



In [8]:
#restaurantes
!wget https://data-architect-test-source.s3-sa-east-1.amazonaws.com/restaurant.csv.gz -O restaurant.csv.gz

--2025-06-17 20:46:54--  https://data-architect-test-source.s3-sa-east-1.amazonaws.com/restaurant.csv.gz
Resolving data-architect-test-source.s3-sa-east-1.amazonaws.com (data-architect-test-source.s3-sa-east-1.amazonaws.com)... 3.5.234.207, 3.5.232.137, 16.12.2.14, ...
Connecting to data-architect-test-source.s3-sa-east-1.amazonaws.com (data-architect-test-source.s3-sa-east-1.amazonaws.com)|3.5.234.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 403165 (394K) [text/csv]
Saving to: ‘restaurant.csv.gz’

restaurant.csv.gz   100%[===================>] 393.72K   576KB/s    in 0.7s    

2025-06-17 20:46:56 (576 KB/s) - ‘restaurant.csv.gz’ saved [403165/403165]



In [9]:
#arquivo A/B test
!wget https://data-architect-test-source.s3-sa-east-1.amazonaws.com/ab_test_ref.tar.gz -O ab_test_ref.tar.gz
!tar -xzvf ab_test_ref.tar.gz

--2025-06-17 20:47:17--  https://data-architect-test-source.s3-sa-east-1.amazonaws.com/ab_test_ref.tar.gz
Resolving data-architect-test-source.s3-sa-east-1.amazonaws.com (data-architect-test-source.s3-sa-east-1.amazonaws.com)... 16.12.1.62, 16.12.0.58, 3.5.232.102, ...
Connecting to data-architect-test-source.s3-sa-east-1.amazonaws.com (data-architect-test-source.s3-sa-east-1.amazonaws.com)|16.12.1.62|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31225850 (30M) [application/x-gzip]
Saving to: ‘ab_test_ref.tar.gz’

ab_test_ref.tar.gz  100%[===================>]  29.78M  10.2MB/s    in 2.9s    

2025-06-17 20:47:21 (10.2 MB/s) - ‘ab_test_ref.tar.gz’ saved [31225850/31225850]

._ab_test_ref.csv
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
ab_test_ref.csv


In [10]:
#pedidos
!wget https://data-architect-test-source.s3-sa-east-1.amazonaws.com/order.json.gz -O order.json.gz

--2025-06-17 20:47:40--  https://data-architect-test-source.s3-sa-east-1.amazonaws.com/order.json.gz
Resolving data-architect-test-source.s3-sa-east-1.amazonaws.com (data-architect-test-source.s3-sa-east-1.amazonaws.com)... 3.5.232.230, 3.5.234.115, 3.5.234.11, ...
Connecting to data-architect-test-source.s3-sa-east-1.amazonaws.com (data-architect-test-source.s3-sa-east-1.amazonaws.com)|3.5.232.230|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1610800545 (1.5G) [application/json]
Saving to: ‘order.json.gz’

order.json.gz       100%[===================>]   1.50G  17.4MB/s    in 87s     

2025-06-17 20:49:07 (17.7 MB/s) - ‘order.json.gz’ saved [1610800545/1610800545]



In [16]:
#consumidores
consumers_df = spark.read.csv("consumer.csv.gz", header=True, inferSchema=True)

#restaurantes
restaurants_df = spark.read.csv("restaurant.csv.gz", header=True, inferSchema=True)

#pedidos
orders_df = spark.read.json("order.json.gz")

#A/B test
import os
[print(f) for f in os.listdir() if f.endswith(".csv")]
abtest_df = spark.read.csv("ab_test_ref.csv", header=True, inferSchema=True)

._ab_test_ref.csv
ab_test_ref.csv


In [17]:
from pyspark.sql.functions import col, countDistinct, avg, sum, count

#agrupa por customer_id e calcula metricas dos pedidos
user_orders_df = orders_df.groupBy("customer_id").agg(
    countDistinct("order_id").alias("total_orders"),
    sum("order_total_amount").alias("total_spent"),
    avg("order_total_amount").alias("avg_order_value"),
    count("order_id").alias("order_count")
)

In [19]:
#junta consumidores+grupo teste
consumers_ab_df = consumers_df.join(abtest_df, on="customer_id", how="inner")

#junta com as métricas de pedido
full_df = consumers_ab_df.join(user_orders_df, on="customer_id", how="left")

#preenche 0 para usuários que não fizeram pedidos (inves de null)
from pyspark.sql.functions import lit, coalesce, when

full_df = full_df.fillna({
    "total_orders": 0,
    "total_spent": 0.0,
    "avg_order_value": 0.0,
    "order_count": 0
})

#cria flag de retenção simples: 1 se fez 2+ pedidos
full_df = full_df.withColumn("is_retained", when(col("total_orders") >= 2, 1).otherwise(0))

In [20]:
full_df.select("customer_id", "is_target", "total_orders", "total_spent", "avg_order_value", "is_retained").show(10)

+--------------------+---------+------------+-----------+------------------+-----------+
|         customer_id|is_target|total_orders|total_spent|   avg_order_value|is_retained|
+--------------------+---------+------------+-----------+------------------+-----------+
|00006f567cb362ba9...|   target|           5|      533.1|            106.62|          1|
|0000c21984ae00cef...|  control|          23|    1539.92| 45.29176470588236|          1|
|0001226e517517758...|   target|           2|      173.0|             43.25|          1|
|00021cd56b6d6c980...|   target|           3|       79.7|15.940000000000001|          1|
|00021f6dc15d10418...|   target|           2|      220.6| 73.53333333333333|          1|
|0002287b123ac1afc...|   target|           1|      107.6|              53.8|          0|
|00022b8c0c7af061f...|  control|           5|      115.8|              19.3|          1|
|00024bc2f09ce5769...|  control|           1|     221.56|            110.78|          0|
|00027035d16a4de43...

In [21]:
#separando os dfs por grupo
grupo_controle = full_df.filter(full_df.is_target == "control")
grupo_teste = full_df.filter(full_df.is_target == "target")


In [22]:
#indicadores

from pyspark.sql.functions import avg

#ticket medio
controle_avg_ticket = grupo_controle.agg(avg("avg_order_value")).first()[0]
teste_avg_ticket = grupo_teste.agg(avg("avg_order_value")).first()[0]

#retenção media
controle_retencao = grupo_controle.agg(avg("is_retained")).first()[0]
teste_retencao = grupo_teste.agg(avg("is_retained")).first()[0]

#total medio gasto
controle_total = grupo_controle.agg(avg("total_spent")).first()[0]
teste_total = grupo_teste.agg(avg("total_spent")).first()[0]

#numero medio de pedidos
controle_pedidos = grupo_controle.agg(avg("total_orders")).first()[0]
teste_pedidos = grupo_teste.agg(avg("total_orders")).first()[0]

print(f"Ticket médio - Controle: R${controle_avg_ticket:.2f} - Teste: R${teste_avg_ticket:.2f}")
print(f"Retenção - Controle: {controle_retencao*100:.2f}% - Teste: {teste_retencao*100:.2f}%")
print(f"Total gasto - Controle: R${controle_total:.2f} - Teste: R${teste_total:.2f}")
print(f"Nº médio pedidos - Controle: {controle_pedidos:.2f} - Teste: {teste_pedidos:.2f}")

🎯 Ticket médio - Controle: R$48.01 | Teste: R$47.83
🔁 Retenção - Controle: 47.63% | Teste: 57.62%
💸 Total gasto - Controle: R$202.67 | Teste: R$228.77
📦 Nº médio pedidos - Controle: 2.80 | Teste: 3.18


In [25]:
# converte pra pandar pra fazer teste estatistico
controle_pd = grupo_controle.select("is_retained", "avg_order_value", "total_spent").toPandas()
teste_pd = grupo_teste.select("is_retained", "avg_order_value", "total_spent").toPandas()

In [26]:
#testa se a diferença entre os grupos é significativa

from scipy.stats import ttest_ind

#teste de diferença no ticket medio
t_stat, p_val = ttest_ind(controle_pd["avg_order_value"].dropna(),
                          teste_pd["avg_order_value"].dropna(),
                          equal_var=False)

print(f"Teste T - Ticket médio | p-valor: {p_val:.4f} {'Significativo' if p_val < 0.05 else 'Não significativo'}")

📊 Teste T - Ticket médio | p-valor: 0.4213 ❌ Não significativo


In [27]:
#teste de significância da taxa de retenção

t_stat_retencao, p_val_retencao = ttest_ind(
    controle_pd["is_retained"],
    teste_pd["is_retained"],
    equal_var=False
)

print(f"Teste T - Retenção | p-valor: {p_val_retencao:.4f} {'Significativo' if p_val_retencao < 0.05 else 'Não significativo'}")

🧪 Teste T - Retenção | p-valor: 0.0000 ✅ Significativo


In [29]:
#segmentacao ativo/inativo
from pyspark.sql.functions import when

full_df = full_df.withColumn(
    "segmento_atividade",
    when(col("total_orders") == 0, "inativo").otherwise("ativo")
)

In [30]:
#segmentacao de ticket alto/baixo
ticket_median = full_df.select("avg_order_value").toPandas()["avg_order_value"].median()

full_df = full_df.withColumn(
    "segmento_ticket",
    when(col("avg_order_value") >= ticket_median, "ticket_alto").otherwise("ticket_baixo")
)

In [32]:
#retençao por segmento de atividade

full_df.groupBy("segmento_atividade", "is_target").agg(
    avg("is_retained").alias("retencao_media"),
    avg("total_spent").alias("gasto_medio"),
    avg("total_orders").alias("pedidos_medios")
).show()

+------------------+---------+------------------+------------------+-----------------+
|segmento_atividade|is_target|    retencao_media|       gasto_medio|   pedidos_medios|
+------------------+---------+------------------+------------------+-----------------+
|             ativo|   target| 0.576159356400437| 228.7713499931779|3.177162625100114|
|             ativo|  control|0.4762591804402172|202.67395868076323|2.803428289212653|
+------------------+---------+------------------+------------------+-----------------+



In [33]:
#retenção por segmento de ticket

full_df.groupBy("segmento_ticket", "is_target").agg(
    avg("is_retained").alias("retencao_media"),
    avg("total_spent").alias("gasto_medio"),
    avg("total_orders").alias("pedidos_medios")
).show()

+---------------+---------+------------------+------------------+------------------+
|segmento_ticket|is_target|    retencao_media|       gasto_medio|    pedidos_medios|
+---------------+---------+------------------+------------------+------------------+
|    ticket_alto|   target|0.5946436697509659| 319.2006203099988| 3.278162447690345|
|    ticket_alto|  control|0.4969688887407136|285.45441119762904|  2.90403796251535|
|   ticket_baixo|   target|0.5574624205201334|137.30188414103168| 3.075001015356706|
|   ticket_baixo|  control|0.4556044467596954| 120.1132496702578|2.7030856876849585|
+---------------+---------+------------------+------------------+------------------+

